# Convolutional Neural Network for ِArabic Handwritten Characters dataset classification

Arabic Handwritten Characters dataset
    
Arabic Handwritten Characters Recognition Using Convolutional Neural Network
Ahmed El-Sawy, Mohamed Loey, Hazem EL-Bakry
WSEAS Transactions on Computer Research, ISSN / E-ISSN: 1991-8755 / 2415-1521, Volume 5, 2017, Art. 2, pp. 11-19



In [4]:
pip install tflearn


  Created wheel for tflearn: filename=tflearn-0.5.0-py3-none-any.whl size=127300 sha256=1f351d375f61a1a1eeda3fb02b1428432d77d3a1d9e0f7a7aa5ef488d38b7827
  Stored in directory: c:\users\mans\appdata\local\pip\cache\wheels\65\9b\15\cb1e6b279c14ed897530d15cfd7da8e3df8a947e593f5cfe59
Successfully built tflearn
Note: you may need to restart the kernel to use updated packages.


In [5]:
from __future__ import division, print_function, absolute_import

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tflearn
from tflearn.data_utils import to_categorical
import tflearn.data_utils as du
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [ ]:
# read training & testing data

trainx = pd.read_csv("arabic letters/csvTrainImages 13440x1024.csv",header=None)
trainy = pd.read_csv("arabic letters/csvTrainLabel 13440x1.csv",header=None)

testx = pd.read_csv("arabic letters/csvTestImages 3360x1024.csv",header=None)
testy = pd.read_csv("arabic letters/csvTestLabel 3360x1.csv",header=None)

In [ ]:
trainx.head()

In [5]:
# Split data into training set and validation set
#training images
trainx = trainx.values.astype('float32')
#training labels
trainy = trainy.values.astype('int32')-1

#testing images
testx = testx.values.astype('float32')
#testing labels
testy = testy.values.astype('int32')-1

In [6]:
trainx[0]

In [7]:
#One Hot encoding of train labels.
trainy = to_categorical(trainy,28)

#One Hot encoding of test labels.
testy = to_categorical(testy,28)

In [8]:
trainy[27]

In [9]:
print(trainx.shape, trainy.shape, testx.shape, testy.shape)

In [10]:
# reshape input images to 28x28x1
trainx = trainx.reshape([-1, 32, 32, 1])
testx = testx.reshape([-1, 32, 32, 1])

In [11]:
print(trainx.shape, trainy.shape, testx.shape, testy.shape)

In [12]:
#Zero center every sample with specified mean. If not specified, the mean is evaluated over all samples.
trainx, mean1 = du.featurewise_zero_center(trainx)
testx, mean2 = du.featurewise_zero_center(testx)

In [13]:
print(trainx.shape, trainy.shape, testx.shape, testy.shape)

In [14]:
trainx[0]

In [15]:
# Building convolutional network
network = input_data(shape=[None, 32, 32, 1], name='input')
network = conv_2d(network, 80, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = conv_2d(network, 64, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2)
network = local_response_normalization(network)
network = fully_connected(network, 1024, activation='relu')
network = dropout(network, 0.8)
network = fully_connected(network, 512, activation='relu')
network = dropout(network, 0.8)
network = fully_connected(network, 28, activation='softmax')
network = regression(network, optimizer='sgd', learning_rate=0.01,
                     loss='categorical_crossentropy', name='target')

In [16]:
#model complile
model = tflearn.DNN(network, tensorboard_verbose=0)

In [17]:
#model fitting
model.fit({'input': trainx}, {'target': trainy}, n_epoch=30,
           validation_set=({'input': testx}, {'target': testy}),
           snapshot_step=100, show_metric=True, run_id='convnet_arabic_digits')

In [18]:
# Evaluate model
score = model.evaluate(testx, testy)
print('Test accuarcy: %0.2f%%' % (score[0] * 100))